In [1]:
%reload_ext autoreload
%autoreload 2
from recsys.MovieLens import MovieLens
from surprise import SVD

In [2]:
def BuildAntiTestSetForUser(testSubject, trainset):
    fill = trainset.global_mean

    anti_testset = []
    
    u = trainset.to_inner_uid(str(testSubject))
    
    user_items = set([j for (j, _) in trainset.ur[u]])
    anti_testset += [(trainset.to_raw_uid(u), trainset.to_raw_iid(i), fill) for
                             i in trainset.all_items() if
                             i not in user_items]
    return anti_testset

In [3]:
# Pick an arbitrary test subject
testSubject = 85

lens = MovieLens()

print("Loading movie ratings...")
data = lens.loadMovieLensLatestSmall()

userRatings = lens.getUserRatings(testSubject)
loved = []
hated = []
for ratings in userRatings:
    if (float(ratings[1]) > 4.0):
        loved.append(ratings)
    if (float(ratings[1]) < 3.0):
        hated.append(ratings)

print("\nUser ", testSubject, " loved these movies:")
for ratings in loved:
    print(lens.getMovieName(ratings[0]))
print("\n...and didn't like these movies:")
for ratings in hated:
    print(lens.getMovieName(ratings[0]))

Loading movie ratings...

User  85  loved these movies:
Jumanji (1995)
GoldenEye (1995)
Braveheart (1995)
Jerky Boys, The (1995)
LÃ©on: The Professional (a.k.a. The Professional) (LÃ©on) (1994)
Pulp Fiction (1994)
Stargate (1994)
Shawshank Redemption, The (1994)
Star Trek: Generations (1994)
Clear and Present Danger (1994)
Speed (1994)
True Lies (1994)
Fugitive, The (1993)
Jurassic Park (1993)
Terminator 2: Judgment Day (1991)
Mission: Impossible (1996)
Rock, The (1996)

...and didn't like these movies:
Grumpier Old Men (1995)
Mortal Kombat (1995)
Postman, The (Postino, Il) (1994)
Casper (1995)
Lord of Illusions (1995)
Mighty Morphin Power Rangers: The Movie (1995)
Prophecy, The (1995)
Dolores Claiborne (1995)
Heavenly Creatures (1994)
Little Women (1994)
Miracle on 34th Street (1994)
Nell (1994)
Poison Ivy II (1996)
Tank Girl (1995)
While You Were Sleeping (1995)
Wes Craven's New Nightmare (Nightmare on Elm Street Part 7: Freddy's Finale, A) (1994)
Naked Gun 33 1/3: The Final Insult (

In [4]:
# Building recommendation model
trainSet = data.build_full_trainset()

algo = SVD()
algo.fit(trainSet)

In [5]:
# Computing recommendations
testSet = BuildAntiTestSetForUser(testSubject, trainSet)
predictions = algo.test(testSet)

recommendations = []

print ("\nWe recommend:")
for userID, movieID, actualRating, estimatedRating, _ in predictions:
    intMovieID = int(movieID)
    recommendations.append((intMovieID, estimatedRating))

recommendations.sort(key=lambda x: x[1], reverse=True)

for ratings in recommendations[:10]:
    print(lens.getMovieName(ratings[0]))


We recommend:
Charade (1963)
African Queen, The (1951)
Star Wars: Episode V - The Empire Strikes Back (1980)
Inglourious Basterds (2009)
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
Princess Bride, The (1987)
Best Years of Our Lives, The (1946)
Fall, The (2006)
Third Man, The (1949)
It's a Wonderful Life (1946)
